In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import os
os.chdir('/home/himanshu/cats_dogs/')

random.seed(42)

In [3]:
## Obtaining image file names from local directory
IMAGE_DIR_PATH = '/home/himanshu/cats_dogs/train'
image_paths = [os.path.join(IMAGE_DIR_PATH, x) for x in os.listdir(IMAGE_DIR_PATH) if x.endswith('.jpg')]

In [4]:
len(image_paths)

25000

In [5]:
## Obtaining labels for the whole data set of 25000 images
labels = []
for x in os.listdir(IMAGE_DIR_PATH):
    if x.startswith('cat'):
        label = 0
    elif x.startswith('dog'):
        label = 1
        
    labels.append(label)

In [6]:
len(labels)

25000

In [7]:
## Splitting the training and validation dataset

x_train, x_cv, y_train, y_cv = train_test_split(pd.DataFrame(image_paths), pd.DataFrame(labels),test_size = 0.2, random_state = 42)

In [8]:
## Paths for Training Images
x_path = np.asarray(x_train[:8192]).reshape(8192,)
y_path = np.asarray(y_train[:8192]).reshape(8192,1)

In [9]:
print 'x_train = %s, y_train = %s' %(x_path.shape,y_path.shape)

x_train = (8192,), y_train = (8192, 1)


In [10]:
## Paths for Validation Images
x_cv = np.asarray(x_cv[:1000]).reshape(len(x_cv[:1000]),)
y_cv = np.asarray(y_cv[:1000]).reshape(len(y_cv[:1000]),1)

In [11]:
print 'x_cv = %s, y_cv = %s' %(x_cv.shape,y_cv.shape)

x_cv = (1000,), y_cv = (1000, 1)


#### Defining training and network parameters

In [12]:
## Training Parameters 
learning_rate = 0.0001
num_epochs = 1
w = 64      ## height and width of image
l = 3       ## num of channels in input image
batch_size = 64
display_step = 10
# epsilon = 10e-8

## Network Parameters
num_classes = 2     # Num of classes in data
dropout = 0.40        # Dropout, probability to keep units

## tf Graph input
# X = tf.placeholder(tf.float32, shape = [None,w,w,l])
# Y = tf.placeholder(tf.float32, [None, num_classes])
# phase = tf.placeholder(tf.bool, name='phase')
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

#### Defining augmentation and data parsing functions

In [13]:
def parse_data(image_paths, label):
    image_content = tf.read_file(image_paths)
    images = tf.image.decode_jpeg(image_content, channels=3)
    return images, label

def bright(image, label):
    cond_brightness = tf.cast(tf.random_uniform([], maxval=2, dtype=tf.int32), tf.bool)
    br_image = tf.cond(cond_brightness, lambda: tf.image.random_hue(image, 0.1), lambda: tf.identity(image))
    return br_image, label

def contrast(image, label):
    cond_contrast = tf.cast(tf.random_uniform([], maxval=2, dtype=tf.int32), tf.bool)
    con_image = tf.cond(cond_contrast, lambda: tf.image.random_contrast(image, 0.2, 1.8), lambda: tf.identity(image))
    return con_image, label

def flipping(image, label):
    seed = random.random()
    flip_image = tf.image.random_flip_left_right(image, seed=seed)
    return flip_image, label

def preprocess(image,label):
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = tf.image.resize_images(image, [w, w])
    return image,label

#### Creating batch input data for training

In [14]:
def batch_input(x_path, y_path, Augment=False, num_threads=4):
    
    ## Creating a tensor of image paths
    img_path_tensors = tf.constant(x_path)
    label_tensors = tf.constant(y_path)
    
    ## Creating dataset out of the tensor file
    path_data = tf.data.Dataset.from_tensor_slices((img_path_tensors,label_tensors))
    
    ## Parsing the dataset
    data = path_data.map(parse_data, num_parallel_calls=num_threads)
    
    ## Normalizing and resizing the training data
    data = data.map(preprocess, num_parallel_calls=num_threads)  
    
    ## If augmentation is to be applied
    if Augment:
        br_data = data.map(bright, num_parallel_calls=num_threads)      ## Brightness augmentation
        
        con_data = data.map(contrast, num_parallel_calls=num_threads)   ## Contrast augmentation
        
        flip_data = data.map(flipping, num_parallel_calls=num_threads)  ## Flip augmentation
        
        ## Concatenating the original and augmented data
        train_data = data.concatenate(br_data)
        train_data = train_data.concatenate(con_data)
        train_data = train_data.concatenate(flip_data)
    
    else:
        train_data = data
    
    train_data = train_data.shuffle(32768)
    
    X = train_data.batch(batch_size).prefetch(1)
    
    ## Create iterator
    iterator = X.make_initializable_iterator()

    ## Next element Op
    x_batch, y_batch = iterator.get_next()

    ## Data set init. op
    init_op = iterator.make_initializer(X)

    return x_batch, y_batch, init_op

#### Defining the convolution network architecture 

In [15]:
def conv_net(x_batch):
    conv1 = tf.layers.conv2d(x_batch, 16, 3, padding = 'SAME', activation=tf.nn.relu)

    conv2 = tf.layers.conv2d(conv1, 32, 3, padding = 'SAME', activation=tf.nn.relu)
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)   ## Max-pool layer(2x2, s2)

    ## Fully connected layers
    ## Reshaping the output from 3D image shape to 1D vectors for fully connected layers
    max_pool = tf.layers.flatten(conv2)
    fc1 = tf.layers.dense(max_pool, 4096, activation=tf.nn.relu)
    fc1 = tf.nn.dropout(fc1, dropout)         ## Applying Dropout(keep_prob = 0.4)

    fc2 = tf.layers.dense(fc1, 512, activation=tf.nn.relu)
    fc2 = tf.nn.dropout(fc2, dropout)         ## Applying Dropout(keep_prob = 0.4)

    logits = tf.layers.dense(fc2, 2, activation=tf.nn.relu)

    ## Obtaining predictions using softmax layer
    prediction = tf.nn.softmax(logits)

    return logits, prediction

In [16]:
## Obtaining the batches for input 
x_batch, y_batch, init_op = batch_input(x_path, y_path, Augment=True, num_threads=4)

## Forward propagation in conv_net to obtain logits and predictions
logits, prediction = conv_net(x_batch)

## Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_batch))
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

## Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_batch, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

tf.summary.scalar('accuracy',accuracy)
tf.summary.scalar('loss',loss_op)
merged = tf.summary.merge_all()
    
writer = tf.summary.FileWriter('/home/himanshu/cats_dogs/', tf.get_default_graph())

In [17]:
## Forward prop for validation dataset
x_cv_batch, y_cv_batch, val_init_op = batch_input(x_cv, y_cv, Augment=False, num_threads=4)
val_log, val_pred = conv_net(x_cv_batch)

## Define loss and 
val_loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=val_log, labels=y_cv_batch))

## Evaluate model
correct_pred = tf.equal(tf.argmax(val_pred, 1), tf.argmax(y_cv_batch, 1))
val_accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
""""tf.set_random_seed(42)
## Initializing layer weights & biases
weights = {
    # 3x3 conv, 1 input, 64 outputs
    'W1': tf.Variable(tf.random_normal([3, 3, 3, 64], mean=0, stddev=0.1), name='W1'),
    # 3x3 conv, 64 inputs, 64 outputs
    'W2': tf.Variable(tf.random_normal([3, 3, 64, 64], mean=0, stddev=0.1), name='W2'),
    # fully connected, 32*32*64 inputs, 4096 outputs
    'fw1': tf.Variable(tf.random_normal([32*32*64, 4096]),name='fw1'),
    # fully connected, 4096 inputs, 1000 outputs
    'fw2': tf.Variable(tf.random_normal([4096, 1000]),name='fw2'),
    # 1000 inputs, 2 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1000, num_classes]),name='out')
}

biases = {
    'b1': tf.Variable(tf.random_normal([64]),name='b1'),
    'b2': tf.Variable(tf.random_normal([64]),name='b2'),
    'fb1': tf.Variable(tf.random_normal([4096]),name='fb1'),
    'fb2': tf.Variable(tf.random_normal([1000]),name='fb2'),
    'out': tf.Variable(tf.random_normal([num_classes]),name='out')
}""""



#### Main Initiation Function 

In [18]:
## Start training
with tf.Session() as sess:

 
    ## Run the initializer
    sess.run(init_op)
    sess.run(init)
    
    for epoch in range(num_epochs):
        for step in range(1, int((32768/batch_size)+1)):
            ## Run optimization op (backprop)
            sess.run(train_op)
            if step % display_step == 0 or step == 1:
                ## Calculate batch loss and accuracy
                loss, acc,summary = sess.run([loss_op,accuracy,merged])
                writer.add_summary(summary,step)
                print("Epoch " + str(epoch)+", Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))

                ##saving checkpoints after every 10 iterations
                save_path = tf.train.Saver().save(sess, '/home/himanshu/cats_dogs/chkpts.ckpt', global_step=step)
    
    sess.run(val_init_op)
    t_acc=[]
    t_loss=[]
    for t_step in range(1, int((1000/batch_size)+1)):
        ## Calculate val batch loss and accuracy
        tmp_loss, tmp_acc = sess.run([val_loss_op,val_accuracy])
        t_loss.append(tmp_loss)
        t_acc.append(tmp_acc)
    ## Calculate accuracy for 1000 validation images
    print("Cross Val Accuracy "+ "{:.4f}".format(sum(t_acc)/len(t_acc)) + \
          ", Cross Val loss "+ "{:.4f}".format(sum(t_loss)/len(t_loss)))

writer.close()

Epoch 0, Step 1, Minibatch Loss= 0.7707, Training Accuracy= 0.484
Epoch 0, Step 10, Minibatch Loss= 0.7673, Training Accuracy= 0.938
Epoch 0, Step 20, Minibatch Loss= 0.7581, Training Accuracy= 1.000
Epoch 0, Step 30, Minibatch Loss= 0.7365, Training Accuracy= 1.000
Epoch 0, Step 40, Minibatch Loss= 0.7148, Training Accuracy= 1.000
Epoch 0, Step 50, Minibatch Loss= 0.6498, Training Accuracy= 1.000
Epoch 0, Step 60, Minibatch Loss= 0.6931, Training Accuracy= 1.000
Epoch 0, Step 70, Minibatch Loss= 0.6282, Training Accuracy= 1.000
Epoch 0, Step 80, Minibatch Loss= 0.8448, Training Accuracy= 1.000
Epoch 0, Step 90, Minibatch Loss= 0.6498, Training Accuracy= 1.000
Epoch 0, Step 100, Minibatch Loss= 0.7148, Training Accuracy= 1.000
Epoch 0, Step 110, Minibatch Loss= 0.6282, Training Accuracy= 1.000
Epoch 0, Step 120, Minibatch Loss= 0.7798, Training Accuracy= 1.000
Epoch 0, Step 130, Minibatch Loss= 0.5415, Training Accuracy= 1.000
Epoch 0, Step 140, Minibatch Loss= 0.6715, Training Accurac

OutOfRangeError: End of sequence
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?,64,64,3], [?,1]], output_types=[DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator)]]

Caused by op u'IteratorGetNext', defined at:
  File "/home/himanshu/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/himanshu/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/himanshu/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/himanshu/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/himanshu/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/himanshu/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py"